#### Practice by Saad Asghar Ali

Credits: Indiantechwarrior

In [1]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip

In [2]:
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.utils import to_categorical

In [3]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [4]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt') # to pull y_train and y_test.txt
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'C:/Users/MST/Documents/SAAD ASGHAR ALI/WORK/Work - Fiverr/saurabh/DATA SET/UCI HAR/UCI HAR Dataset/UCI HAR Dataset/')
	print('Shape of trainX and trainY:\n', trainX.shape, trainy.shape)
	#print(trainy[:20],trainy[-20:]) # line by me # = 5,2
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'C:/Users/MST/Documents/SAAD ASGHAR ALI/WORK/Work - Fiverr/saurabh/DATA SET/UCI HAR/UCI HAR Dataset/UCI HAR Dataset/')
	print('Shape of testX and testY:\n', testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print('doing zero-offset and onehotencod on Y')
	print('Final Shape of train and test:\n', trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [5]:
# load data
trainX, trainy, testX, testy = load_dataset()

Shape of trainX and trainY:
 (7352, 128, 9) (7352, 1)
Shape of testX and testY:
 (2947, 128, 9) (2947, 1)
doing zero-offset and onehotencod on Y
Final Shape of train and test:
 (7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [6]:
# dtype of X data is float64, Y data is float32
# Type of all the data is numpy.ndarray
#trainX[0].shape # =(128,9)
# print(trainX[0][0].round(2))
import numpy as np
#np.amax(trainX) # =5.75  #np.amin(trainX) # =-5.97  #np.mean(trainX) # =0.1
#np.amax(testX) # =3.47   # np.amin(testX) # =-3.43   # np.mean(testX) # =0.1
trainy[0]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

In [7]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
print(n_timesteps, n_features, n_outputs)

128 9 6


In [8]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 10, 32
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
	model.add(Dropout(0.5))
	model.add(MaxPool1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [10]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=3):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# run the experiment
run_experiment()

Shape of trainX and trainY:
 (7352, 128, 9) (7352, 1)
Shape of testX and testY:
 (2947, 128, 9) (2947, 1)
doing zero-offset and onehotencod on Y
Final Shape of train and test:
 (7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 91.449
>#2: 92.535
>#3: 90.058
[91.44893288612366, 92.53478050231934, 90.05768299102783]
Accuracy: 91.347% (+/-1.014)


##### NICE ACCURACY OF 91.3%

In [ ]:
# Problem: Low visibility of the output
# Solution: I just solved this problem by installing gast:
#pip install --user gast==0.2.2

#### CONGRATULATIONS!
##### Practice done